# provider.py Module

Handles provider registration, dashboard, profile, and calendar/event logic.

In [ ]:
# Example: Provider registration and profile logic
# (Add actual functions from app.py as needed)
def register_provider(...):
    pass

def get_provider_profile(provider_id):
    pass

# Example: Calendar/event logic
def get_provider_events(provider_id):
    pass


# Provider Module
All provider dashboard, service/product management, analytics, and related logic from the Flask app.

In [ ]:
from flask import session, request, render_template, redirect, url_for, flash
from contextlib import closing
import os
import datetime

# --- Provider Dashboard ---
def provider_dashboard():
    current_provider_id = session.get('provider_id', 0)
    with closing(get_db()) as db:
        cur = db.cursor()
        cur.execute("SELECT COUNT(*) FROM leads WHERE provider_id = ?", (current_provider_id,))
        leads_count = cur.fetchone()[0]
        cur.execute("SELECT COUNT(*) FROM services WHERE provider_id = ?", (current_provider_id,))
        services_count = cur.fetchone()[0]
        cur.execute("SELECT COUNT(*) FROM products WHERE provider_id = ?", (current_provider_id,))
        products_count = cur.fetchone()[0]
        total_views = (services_count + products_count) * 15  # Mock data
    return render_template("provider_dashboard.html", 
                         leads_count=leads_count, 
                         services_count=services_count,
                         products_count=products_count,
                         total_views=total_views,
                         title="Provider Dashboard")


In [ ]:
# --- Provider Services Management ---
def provider_manage_services():
    current_provider_id = session.get('provider_id', 0)
    with closing(get_db()) as db:
        cur = db.cursor()
        cur.execute(
            "SELECT id, title, description, price, active, is_certified, certification_proof, created_at FROM services WHERE provider_id = ? ORDER BY created_at DESC",
            (current_provider_id,)
        )
        services = cur.fetchall()
    return render_template("provider_services.html", services=services, title="Manage My Services")

def provider_service_new():
    if request.method == "POST":
        title = request.form.get("title", "").strip()
        description = request.form.get("description", "").strip()
        price = request.form.get("price", "").strip()
        active = 1 if request.form.get("active") == "on" else 0
        is_certified = 1 if request.form.get("is_certified") == "on" else 0
        certification_proof = request.form.get("certification_proof", "").strip()
        current_provider_id = session.get('provider_id', 0)
        if not title:
            flash("Title is required.", "error")
            return render_template("provider_service_form.html", form=request.form, mode="new", title="Add Service")
        with closing(get_db()) as db:
            cur = db.cursor()
            cur.execute("SELECT id FROM services WHERE provider_id = ? AND LOWER(title) = LOWER(?) AND active = 1", (current_provider_id, title))
            existing_service = cur.fetchone()
            if existing_service:
                flash(f"You already have a service called '{title}'. Please update your existing service instead of creating a duplicate.", "error")
                return render_template("provider_service_form.html", form=request.form, mode="new", title="Add Service")
        if is_certified and not certification_proof:
            flash("Certification proof/link is required for certified services.", "error")
            return render_template("provider_service_form.html", form=request.form, mode="new", title="Add Service")
        with closing(get_db()) as db:
            cur = db.cursor()
            # Get provider info for posted_by field
            if current_provider_id > 0:
                cur.execute("SELECT first_name, business_name FROM providers WHERE id = ?", (current_provider_id,))
                provider_row = cur.fetchone()
                posted_by = provider_row["first_name"] or provider_row["business_name"] if provider_row else "Provider"
            else:
                cur.execute("SELECT first_name, business_name FROM profile WHERE id = 1")
                profile_row = cur.fetchone()
                posted_by = profile_row["first_name"] or profile_row["business_name"] if profile_row else "Provider"
            cur.execute(
                """
                INSERT INTO services (title, description, price, posted_by, provider_id, active, is_certified, certification_proof, created_at)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                """,
                (title, description, price, posted_by, current_provider_id, active, is_certified, certification_proof, datetime.datetime.utcnow().isoformat())
            )
            db.commit()
        flash("Service added successfully.", "success")
        return redirect(url_for("provider_manage_services"))
    return render_template("provider_service_form.html", form={}, mode="new", title="Add Service")


In [ ]:
def provider_service_edit(service_id):
    current_provider_id = session.get('provider_id', 0)
    with closing(get_db()) as db:
        cur = db.cursor()
        cur.execute("SELECT id, title, description, price, active, is_certified, certification_proof FROM services WHERE id = ? AND provider_id = ?", (service_id, current_provider_id))
        service = cur.fetchone()
        if not service:
            flash("Service not found or access denied.", "error")
            return redirect(url_for("provider_manage_services"))
    if request.method == "POST":
        title = request.form.get("title", "").strip()
        description = request.form.get("description", "").strip()
        price = request.form.get("price", "").strip()
        active = 1 if request.form.get("active") == "on" else 0
        is_certified = 1 if request.form.get("is_certified") == "on" else 0
        certification_proof = request.form.get("certification_proof", "").strip()
        if not title:
            flash("Title is required.", "error")
            return render_template("provider_service_form.html", form=request.form, mode="edit", service=service, title="Edit Service")
        with closing(get_db()) as db:
            cur = db.cursor()
            cur.execute("SELECT id FROM services WHERE provider_id = ? AND LOWER(title) = LOWER(?) AND id != ? AND active = 1", (current_provider_id, title, service_id))
            existing_service = cur.fetchone()
            if existing_service:
                flash(f"You already have another service called '{title}'. Please choose a different title or update that existing service.", "error")
                return render_template("provider_service_form.html", form=request.form, mode="edit", service=service, title="Edit Service")
        if is_certified and not certification_proof:
            flash("Certification proof/link is required for certified services.", "error")
            return render_template("provider_service_form.html", form=request.form, mode="edit", service=service, title="Edit Service")
        with closing(get_db()) as db:
            cur = db.cursor()
            cur.execute(
                """
                UPDATE services
                SET title = ?, description = ?, price = ?, active = ?, is_certified = ?, certification_proof = ?
                WHERE id = ? AND provider_id = ?
                """,
                (title, description, price, active, is_certified, certification_proof, service_id, current_provider_id),
            )
            db.commit()
            flash("Service updated successfully.", "success")
            return redirect(url_for("provider_manage_services"))
    return render_template(
        "provider_service_form.html",
        form=dict(
            title=service["title"],
            description=service["description"],
            price=service["price"],
            active=bool(service["active"]),
            is_certified=service["is_certified"],
            certification_proof=service["certification_proof"],
        ),
        mode="edit",
        service=service,
        title="Edit Service",
    )

def provider_service_delete(service_id):
    current_provider_id = session.get('provider_id', 0)
    with closing(get_db()) as db:
        cur = db.cursor()
        cur.execute("DELETE FROM services WHERE id = ? AND provider_id = ?", (service_id, current_provider_id))
        db.commit()
    flash("Service deleted.", "info")
    return redirect(url_for("provider_manage_services"))


In [ ]:
# --- Provider Products Management ---
def provider_products():
    current_provider_id = session.get('provider_id', 0)
    with closing(get_db()) as db:
        cur = db.cursor()
        cur.execute(
            "SELECT id, title, description, price, active, created_at FROM products WHERE provider_id = ? ORDER BY created_at DESC",
            (current_provider_id,)
        )
        products = cur.fetchall()
    return render_template("provider_products.html", products=products, title="My Products")

def provider_product_new():
    current_provider_id = session.get('provider_id', 0)
    if request.method == "POST":
        title = request.form.get("title", "").strip()
        description = request.form.get("description", "").strip()
        price = request.form.get("price", "").strip()
        active = 1 if request.form.get("active") == "on" else 0
        image_paths = []
        upload_folder = os.path.join(APP_DIR, "static", "uploads", "products")
        os.makedirs(upload_folder, exist_ok=True)
        files = request.files.getlist("images")
        for idx, file in enumerate(files[:5]):
            if file and file.filename:
                ext = os.path.splitext(file.filename)[1].lower()
                if ext not in [".jpg", ".jpeg", ".png", ".gif", ".webp"]:
                    flash(f"Image {idx+1} must be a valid image file.", "error")
                    return render_template("provider_product_form.html", form=request.form, mode="new", title="Add Product")
                filename = f"{current_provider_id}_{int(datetime.datetime.utcnow().timestamp())}_{idx+1}{ext}"
                save_path = os.path.join(upload_folder, filename)
                file.save(save_path)
                image_paths.append(f"/static/uploads/products/{filename}")
            else:
                image_paths.append("")
        while len(image_paths) < 5:
            image_paths.append("")
        if not title:
            flash("Title is required.", "error")
            return render_template("provider_product_form.html", form=request.form, mode="new", title="Add Product")
        with closing(get_db()) as db:
            cur = db.cursor()
            cur.execute(
                """
                INSERT INTO products (title, description, price, provider_id, active, created_at, image1, image2, image3, image4, image5)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """,
                (title, description, price, current_provider_id, active, datetime.datetime.utcnow().isoformat(),
                 image_paths[0], image_paths[1], image_paths[2], image_paths[3], image_paths[4])
            )
            db.commit()
            flash("Product added successfully.", "success")
            return redirect(url_for("provider_products"))
    return render_template("provider_product_form.html", mode="new", title="Add Product")


In [ ]:
def provider_product_edit(product_id):
    current_provider_id = session.get('provider_id', 0)
    with closing(get_db()) as db:
        cur = db.cursor()
        cur.execute("SELECT * FROM products WHERE id = ? AND provider_id = ?", (product_id, current_provider_id))
        product = cur.fetchone()
        if not product:
            flash("Product not found or access denied.", "error")
            return redirect(url_for("provider_products"))
    if request.method == "POST":
        title = request.form.get("title", "").strip()
        description = request.form.get("description", "").strip()
        price = request.form.get("price", "").strip()
        active = 1 if request.form.get("active") == "on" else 0
        image_paths = []
        upload_folder = os.path.join(APP_DIR, "static", "uploads", "products")
        os.makedirs(upload_folder, exist_ok=True)
        for i in range(1, 6):
            remove = request.form.get(f"remove_image{i}")
            file = request.files.get(f"image{i}")
            current_img = product[f"image{i}"] if product else ""
            if remove:
                image_paths.append("")
            elif file and file.filename:
                ext = os.path.splitext(file.filename)[1].lower()
                if ext not in [".jpg", ".jpeg", ".png", ".gif", ".webp"]:
                    flash(f"Image {i} must be a valid image file.", "error")
                    return render_template("provider_product_form.html", form=request.form, mode="edit", product=product, title="Edit Product")
                filename = f"{current_provider_id}_{int(datetime.datetime.utcnow().timestamp())}_{i}{ext}"
                save_path = os.path.join(upload_folder, filename)
                file.save(save_path)
                image_paths.append(f"/static/uploads/products/{filename}")
            else:
                image_paths.append(current_img)
        if not title:
            flash("Title is required.", "error")
            return render_template("provider_product_form.html", form=request.form, mode="edit", product=product, title="Edit Product")
        with closing(get_db()) as db:
            cur = db.cursor()
            cur.execute(
                """
                UPDATE products
                SET title = ?, description = ?, price = ?, active = ?,
                    image1 = ?, image2 = ?, image3 = ?, image4 = ?, image5 = ?
                WHERE id = ? AND provider_id = ?
                """,
                (title, description, price, active,
                 image_paths[0], image_paths[1], image_paths[2], image_paths[3], image_paths[4],
                 product_id, current_provider_id),
            )
            db.commit()
            flash("Product updated successfully.", "success")
            return redirect(url_for("provider_products"))
    return render_template("provider_product_form.html", product=product, mode="edit", title="Edit Product")

def provider_product_delete(product_id):
    current_provider_id = session.get('provider_id', 0)
    with closing(get_db()) as db:
        cur = db.cursor()
        cur.execute("DELETE FROM products WHERE id = ? AND provider_id = ?", (product_id, current_provider_id))
        db.commit()
    flash("Product deleted.", "info")
    return redirect(url_for("provider_products"))


In [ ]:
# --- Provider Analytics ---
def provider_analytics():
    current_provider_id = session.get('provider_id', 0)
    with closing(get_db()) as db:
        cur = db.cursor()
        cur.execute("SELECT COUNT(*) FROM services WHERE provider_id = ? AND active = 1", (current_provider_id,))
        services_count = cur.fetchone()[0]
        cur.execute("SELECT COUNT(*) FROM products WHERE provider_id = ? AND active = 1", (current_provider_id,))
        products_count = cur.fetchone()[0]
        cur.execute("SELECT COUNT(*) FROM leads WHERE provider_id = ?", (current_provider_id,))
        total_leads = cur.fetchone()[0]
        cur.execute("SELECT DATE(created_at) as date, COUNT(*) as count FROM leads WHERE provider_id = ? AND created_at >= date('now', '-30 days') GROUP BY DATE(created_at) ORDER BY date", (current_provider_id,))
        daily_leads = cur.fetchall()
        cur.execute("""
            SELECT s.title, s.price, COUNT(l.id) as lead_count
            FROM services s
            LEFT JOIN leads l ON l.message LIKE '%' || s.title || '%' AND l.provider_id = s.provider_id
            WHERE s.provider_id = ? AND s.active = 1
            GROUP BY s.id, s.title, s.price
            ORDER BY lead_count DESC
            LIMIT 5
        """, (current_provider_id,))
        top_services = cur.fetchall()
        cur.execute("""
            SELECT p.title, p.price, COUNT(l.id) as lead_count
            FROM products p
            LEFT JOIN leads l ON l.message LIKE '%' || p.title || '%' AND l.provider_id = p.provider_id
            WHERE p.provider_id = ? AND p.active = 1
            GROUP BY p.id, p.title, p.price
            ORDER BY lead_count DESC
            LIMIT 5
        """, (current_provider_id,))
        top_products = cur.fetchall()
        cur.execute("""
            SELECT utm_source, SUM(profit) as total_profit, COUNT(*) as visit_count
            FROM visits
            WHERE provider_id = ?
            GROUP BY utm_source
            ORDER BY total_profit DESC
        """, (current_provider_id,))
        traffic_sources = cur.fetchall()
        max_profit = max([row[1] for row in traffic_sources], default=1)
        traffic_chart = [
            {
                "source": row[0],
                "profit": row[1],
                "visits": row[2],
                "bar_width": int((row[1] / max_profit) * 100) if max_profit else 0
            }
            for row in traffic_sources
        ]
    return render_template("provider_analytics.html", 
                             services_count=services_count,
                             products_count=products_count,
                             total_leads=total_leads,
                             total_views=(services_count + products_count) * 15,
                             daily_leads=daily_leads,
                             top_services=top_services,
                             top_products=top_products,
                             traffic_chart=traffic_chart,
                             title="Provider Analytics")
